In [ ]:
# ================================================================
# Notebook 3
# Forecasting Congestion Income — Model Training & Evaluation
# ================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from pathlib import Path
from IPython.display import Markdown, display

print("⚡ Notebook 3 — Forecasting Congestion Income (DK2)")


In [ ]:
# ---------------------------------------------------------------
# 1. Load engineered feature dataset
# ---------------------------------------------------------------

FEATURE_PATH = Path("data/processed/features_congestion_income.parquet")

if not FEATURE_PATH.exists():
    raise FileNotFoundError(
        f"Feature dataset not found at: {FEATURE_PATH}\n"
        "→ Run Notebook 2 first."
    )

df = pd.read_parquet(FEATURE_PATH)

print(f"✔ Loaded feature dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"🕒 Time range: {df.index.min()} → {df.index.max()}")

display(df.head(3))


In [ ]:
# ---------------------------------------------------------------
# 2. Define forecasting horizon
# ---------------------------------------------------------------

HORIZON = 1   # 1-step ahead = 15 minutes
df["target"] = df["RevenueEUR"].shift(-HORIZON)

# Drop final rows where target is missing
df = df.dropna(subset=["target"])

print("✔ Target column created.")
df[["RevenueEUR", "target"]].head(3)


In [ ]:
# ---------------------------------------------------------------
# 3. Train/Test Split (no shuffling!)
# ---------------------------------------------------------------

split_ratio = 0.8
split_idx = int(len(df) * split_ratio)

train = df.iloc[:split_idx].copy()
test  = df.iloc[split_idx:].copy()

X_train = train.drop(columns=["target"])
y_train = train["target"]
X_test  = test.drop(columns=["target"])
y_test  = test["target"]

print(f"✔ Train size: {len(train)} rows")
print(f"✔ Test size:  {len(test)} rows")
print(f"Split at index: {split_idx}")


In [ ]:
# ---------------------------------------------------------------
# 4. Baseline Models
# ---------------------------------------------------------------

def naive_forecast(series):
    return series.shift(1)

def seasonal_naive(series, lag_steps=96):  # 1 day = 96 × 15min
    return series.shift(lag_steps)

def rolling_mean_forecast(series, window=24):  # 6 hours
    return series.rolling(window).mean()

y_pred_naive = naive_forecast(train["target"]).shift(-1).iloc[split_idx:]
y_pred_seasonal = seasonal_naive(train["target"]).shift(-1).iloc[split_idx:]
y_pred_roll = rolling_mean_forecast(train["target"]).shift(-1).iloc[split_idx:]

print("✔ Baselines computed.")


In [ ]:
# ---------------------------------------------------------------
# 5. Evaluation Metrics
# ---------------------------------------------------------------

from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse(y, x):
    return np.sqrt(mean_squared_error(y, x))

def mape(y, x):
    return np.mean(np.abs((y - x) / y)) * 100

def mase(y_true, y_pred, training_series):
    naive = np.abs(training_series.diff()).mean()
    return np.mean(np.abs(y_true - y_pred)) / naive

def evaluate(y_true, y_pred, name):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse_val = rmse(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    mase_val = mase(y_true, y_pred, y_train)

    print(f"\n📊 {name} Performance")
    print(f"MAE:  {mae:,.2f}")
    print(f"RMSE: {rmse_val:,.2f}")
    print(f"MAPE: {mape_val:,.2f}%")
    print(f"MASE: {mase_val:,.2f}")

    return mae, rmse_val, mape_val, mase_val

baseline_results = {}

baseline_results["Naive"] = evaluate(y_test, y_pred_naive, "Naive")
baseline_results["Seasonal Naive"] = evaluate(y_test, y_pred_seasonal, "Seasonal Naive")
baseline_results["Rolling Mean"] = evaluate(y_test, y_pred_roll, "Rolling Mean")


In [ ]:
# ---------------------------------------------------------------
# 6. Random Forest Regressor
# ---------------------------------------------------------------

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

rf_results = evaluate(y_test, y_pred_rf, "Random Forest")


In [ ]:
# ---------------------------------------------------------------
# 7. XGBoost Model
# ---------------------------------------------------------------

try:
    from xgboost import XGBRegressor

    xgb = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.9,
        eval_metric="rmse"
    )

    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)
    xgb_results = evaluate(y_test, y_pred_xgb, "XGBoost")

except ImportError:
    print("⚠️ XGBoost not installed. Skipping this model.")


In [ ]:
# ---------------------------------------------------------------
# 8. Model Comparison Summary
# ---------------------------------------------------------------

results = pd.DataFrame({
    "Naive": baseline_results["Naive"],
    "Seasonal Naive": baseline_results["Seasonal Naive"],
    "Rolling Mean": baseline_results["Rolling Mean"],
    "Random Forest": rf_results,
}, index=["MAE", "RMSE", "MAPE", "MASE"])

display(results)


In [ ]:
# ---------------------------------------------------------------
# 9. Residual Diagnostics
# ---------------------------------------------------------------

residuals = y_test - y_pred_rf

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sns.histplot(residuals, ax=axes[0], kde=True)
axes[0].set_title("Residual Distribution")

sns.scatterplot(x=y_pred_rf, y=residuals, ax=axes[1])
axes[1].set_title("Residual vs Predicted")

sns.lineplot(x=residuals.index, y=residuals, ax=axes[2])
axes[2].set_title("Residual Time Series")

plt.tight_layout()
plt.show()


In [ ]:
# ---------------------------------------------------------------
# 10. True vs Predicted Plot
# ---------------------------------------------------------------

fig = px.line(
    x=y_test.index,
    y=[y_test.values, y_pred_rf],
    labels={"x": "Time", "value": "Congestion Income (EUR)"},
    title="True vs Predicted (Random Forest)"
)

fig.update_layout(
    legend=dict(title="Series", itemsizing="constant"),
    height=400
)

fig.show()


In [ ]:
# ---------------------------------------------------------------
# 11. Feature Importance
# ---------------------------------------------------------------

importances = pd.Series(rf.feature_importances_, index=X_train.columns)
importances = importances.sort_values(ascending=False)

fig = px.bar(
    importances.head(20),
    title="Top 20 Feature Importances (Random Forest)"
)
fig.show()


In [ ]:
# ---------------------------------------------------------------
# 12. Save Model + Forecast Outputs
# ---------------------------------------------------------------

import joblib

MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

joblib.dump(rf, MODEL_DIR / "rf_model.pkl")
y_test.to_frame("true").assign(pred_rf=y_pred_rf).to_parquet(
    Path("data/processed/forecast_results.parquet")
)

print("✔ Model and forecast results saved.")


In [ ]:
# ---------------------------------------------------------------
# 13. LLM-Assisted Forecast Interpretation
# ---------------------------------------------------------------

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.1:8b-instruct-q4_0")

prompt = f"""
You are a senior electricity-market analyst.

Write an interpretive summary of the forecasting model performance:
- how well the model captures daily structure
- when it fails (e.g., spikes, ramps, volatility bursts)
- what the residuals say about model bias
- how feature importance aligns with power-system behavior
- compare model vs naive baselines

Keep it concise and insightful.
"""

try:
    response = requests.post(
        f"{OLLAMA_URL}/api/generate",
        json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
    )
    text = response.json().get("response", "").strip()

    display(Markdown("### 🤖 LLM Forecast Analysis\n" + text))

except Exception as e:
    print("⚠️ LLM unavailable:", e)


## 📘 Notebook 3 Summary — Forecasting

This notebook implemented:
- Horizon definition (15-min ahead)
- Train/test split with time integrity
- Baseline forecasts (naive, seasonal naive, rolling mean)
- ML models (Random Forest, XGBoost)
- Evaluation (MAE, RMSE, MAPE, MASE)
- Residual diagnostics
- Feature importance
- Scenario plots
- LLM-assisted model interpretation
- Model & forecast saving

Proceed to **Notebook 4 — Multi-Step Forecasting & Model Optimization**.

